https://www.kaggle.com/albertovilla/scrapping-ewg-tap-water-database

https://github.com/ravirajuv/Web-Scraping-Walmart-Using-BeautifulSoup/blob/master/Web-Scraping-Using-BeautifulSoup.ipynb

In [1]:
from __future__ import print_function, division
import requests
requests.__path__
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd
import numpy as np
import pickle
import random
import time

In [196]:
name_list, rating_list, link_list = [], [], []

for i in range(0, 3909, 1000):
    # if i == 100:
    #    break
    
    # if i % 20 == 0:
    #    time.sleep(5)
        
    url = 'https://www.ewg.org/skindeep/browse.php?atatime=1000&category=shampoo&&showmore=products&start=' + str(i)
    time.sleep(2 + 0.5*random.randint(1, 10))
    response = requests.get(url)
    print(i, response.status_code)
    
    page = response.text
    soup = BeautifulSoup(page, 'lxml')
    
    for name in soup.find_all(class_ = 'product_name_list'):
        name_list.append(name.find('a').text)
    
    for link in soup.find_all(class_ = 'product_name_list'):    
        link_list.append('https://www.ewg.org' + str((link.find('a')['href'])))

    rating_pic_list = []
    
    for pic in soup.find_all('img'):
        if 'EWGV_SD_Mark' in str(pic) or 'score_image' in str(pic):
            rating_pic_list.append(str(pic))
    
    for pic in rating_pic_list:
        pic = pic.split('/')
        if 'EWGV_SD_Mark.png"' in pic: 
            rating = 1
        else:
            rating = pic[6].split('__')[0]
            rating = rating.split('image')[1]
        rating_list.append(str(rating))

0 200
1000 200
2000 200
3000 200


In [197]:
df = pd.DataFrame({'name': name_list, 'rating': rating_list, 'link': link_list})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3908 entries, 0 to 3907
Data columns (total 3 columns):
name      3908 non-null object
rating    3908 non-null object
link      3908 non-null object
dtypes: object(3)
memory usage: 91.7+ KB


In [199]:
df.head()

,name,rating,link
0,Boyzz Only No Nonsense Hair & Body Wash,1,https://www.ewg.org/skindeep/product/707461/Bo...
1,Soap for Goodness Sake Babassu Shampoo and Bod...,1,https://www.ewg.org/skindeep/product/701926/So...
2,"Soap for Goodness Sake Shampoo and Body Bar, S...",1,https://www.ewg.org/skindeep/product/701976/So...
3,Aromatica B5 + Biotin Fortifying Shampoo,1,https://www.ewg.org/skindeep/product/816883/Ar...
4,Aromatica Cypress Deep Cleansing Shampoo,1,https://www.ewg.org/skindeep/product/833682/Ar...


In [237]:
rand_nums = np.random.randint(1, 3908, 50)

In [243]:
item_dict = defaultdict(int) 

counter = 1

for link in [link_list[i] for i in rand_nums] :
   
    
    response = requests.get(link)
    time.sleep(3 + 0.5*random.randint(1, 10))
    print(response.status_code, counter)
    page = response.text
    soup = BeautifulSoup(page, 'lxml')
        
    item_list = []
    
    for item in soup.find_all(class_ = 'firstcol'):
        try:
            item_list.append(item.find('a').text)
        except:
            pass
    
    # for item in item_list:
    #    if 'search' in item:    
    #        item_list.remove(item)

    # for item in item_list:
    #    if 'http' in item:
    #        item_list.remove(item)
    
    # print(item_list)
    
    for item in item_list:
        item_dict[str(item)] += 1
    
    counter += 1
    

200 1
200 2
200 3
200 4
200 5
200 6
200 7
200 8
200 9
200 10
200 11
200 12
200 13
200 14
200 15
200 16
200 17
200 18
200 19
200 20
200 21
200 22
200 23
200 24
200 25
200 26
200 27
200 28
200 29
200 30
200 31
200 32
200 33
200 34
200 35
200 36
200 37
200 38
200 39
200 40
200 41
200 42
200 43
200 44
200 45
200 46
200 47
200 48
200 49
200 50


In [244]:
item_dict

defaultdict(int,
            {'FRAGRANCE': 39,
             'METHYLISOTHIAZOLINONE': 24,
             'METHYLCHLOROISOTHIAZOLINONE': 22,
             'COCAMIDOPROPYL BETAINE': 26,
             'COCAMIDE MEA': 17,
             'SODIUM LAURETH SULFATE': 28,
             'DIMETHICONE': 14,
             'SODIUM BENZOATE': 21,
             'TETRASODIUM EDTA': 20,
             'SODIUM LAURYL SULFATE': 18,
             'CITRIC ACID': 33,
             'POLYQUATERNIUM 76': 4,
             'TRISODIUM ETHYLENEDIAMINE DISUCCINATE': 9,
             'PANTHENYL ETHYL ETHER': 6,
             'GLYCOL DISTEARATE': 21,
             'SODIUM CITRATE': 14,
             'SODIUM XYLENE SULFONATE': 14,
             'PANTHENOL': 18,
             'WATER': 45,
             'SODIUM CHLORIDE': 30,
             'Amazon.com product page': 20,
             'UNSPECIFIED OILS': 2,
             'ALOE BARBADENSIS (ALOE VERA) LEAF EXTRACT': 6,
             'KERATIN': 2,
             'BUTYROSPERMUM PARKII (SHEA BUTTER) OIL'

In [245]:
common_items_list = []

for key, value in item_dict.items():
    if value > 3:
        common_items_list.append(key)
        
len(common_items_list)

58

In [246]:
for item in common_items_list:
    if 'http' in item:
        common_items_list.remove(item)
        
len(common_items_list)

58

In [247]:
df2 = df.iloc[:50].copy()

In [248]:
for item in common_items_list:
    df2[item] = 0

In [249]:
df2.head()

,name,rating,link,FRAGRANCE,METHYLISOTHIAZOLINONE,METHYLCHLOROISOTHIAZOLINONE,COCAMIDOPROPYL BETAINE,COCAMIDE MEA,SODIUM LAURETH SULFATE,DIMETHICONE,...,SODIUM COCOAMPHOACETATE,HEXYL CINNAMAL,SODIUM HYDROXIDE,MENTHOL,MENTHA PIPERITA (PEPPERMINT) OIL,TEA DODECYLBENZENESULFONATE,DIMETHICONOL,CARBOMER,POLYQUATERNIUM-6,LINALOOL
0,Boyzz Only No Nonsense Hair & Body Wash,1,https://www.ewg.org/skindeep/product/707461/Bo...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Soap for Goodness Sake Babassu Shampoo and Bod...,1,https://www.ewg.org/skindeep/product/701926/So...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Soap for Goodness Sake Shampoo and Body Bar, S...",1,https://www.ewg.org/skindeep/product/701976/So...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Aromatica B5 + Biotin Fortifying Shampoo,1,https://www.ewg.org/skindeep/product/816883/Ar...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aromatica Cypress Deep Cleansing Shampoo,1,https://www.ewg.org/skindeep/product/833682/Ar...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [250]:
for index, row in df2.iterrows():
    
    url = row['link']
    time.sleep(3 + 0.5*random.randint(1, 10))
    response = requests.get(url)
    print(response.status_code, index)
    
    page = response.text
    soup = BeautifulSoup(page, 'lxml')
    
    item_list = []
    
    for item in soup.find_all(class_ = 'firstcol'):
        item_list.append(item.find('a').text)
            
    for item in item_list:
        if item in df2.columns:
            df2.set_value(index, item, 1)

200 0


//anaconda3/envs/metis/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


200 1
200 2
200 3
200 4
200 5
200 6
200 7
200 8
200 9
200 10
200 11
200 12
200 13
200 14
200 15
200 16
200 17
200 18
200 19
200 20
200 21
200 22
200 23
200 24
200 25
200 26
200 27
200 28
200 29
200 30
200 31
200 32
200 33
200 34
200 35
200 36
200 37
200 38
200 39
200 40
200 41
200 42
200 43
200 44
200 45
200 46
200 47
200 48
200 49


In [251]:
df2['sum']= df2.sum(axis=1)

In [252]:
df2.head(20)

,name,rating,link,FRAGRANCE,METHYLISOTHIAZOLINONE,METHYLCHLOROISOTHIAZOLINONE,COCAMIDOPROPYL BETAINE,COCAMIDE MEA,SODIUM LAURETH SULFATE,DIMETHICONE,...,HEXYL CINNAMAL,SODIUM HYDROXIDE,MENTHOL,MENTHA PIPERITA (PEPPERMINT) OIL,TEA DODECYLBENZENESULFONATE,DIMETHICONOL,CARBOMER,POLYQUATERNIUM-6,LINALOOL,sum
0,Boyzz Only No Nonsense Hair & Body Wash,1,https://www.ewg.org/skindeep/product/707461/Bo...,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,7
1,Soap for Goodness Sake Babassu Shampoo and Bod...,1,https://www.ewg.org/skindeep/product/701926/So...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,"Soap for Goodness Sake Shampoo and Body Bar, S...",1,https://www.ewg.org/skindeep/product/701976/So...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,Aromatica B5 + Biotin Fortifying Shampoo,1,https://www.ewg.org/skindeep/product/816883/Ar...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
4,Aromatica Cypress Deep Cleansing Shampoo,1,https://www.ewg.org/skindeep/product/833682/Ar...,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,8
5,ATTITUDE Natural Treatment Shampoo,1,https://www.ewg.org/skindeep/product/669787/AT...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
6,"ATTITUDE Sensitive Skin Shampoo, extra gentle ...",1,https://www.ewg.org/skindeep/product/773165/AT...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
7,"ATTITUDE Sensitive Skin Shampoo, nourish & shi...",1,https://www.ewg.org/skindeep/product/773168/AT...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
8,"ATTITUDE Sensitive Skin Shampoo, repair & colo...",1,https://www.ewg.org/skindeep/product/773166/AT...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
9,"ATTITUDE Sensitive Skin Shampoo, soothing & vo...",1,https://www.ewg.org/skindeep/product/773169/AT...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


In [253]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 62 columns):
name                                               50 non-null object
rating                                             50 non-null object
link                                               50 non-null object
FRAGRANCE                                          50 non-null int64
METHYLISOTHIAZOLINONE                              50 non-null int64
METHYLCHLOROISOTHIAZOLINONE                        50 non-null int64
COCAMIDOPROPYL BETAINE                             50 non-null int64
COCAMIDE MEA                                       50 non-null int64
SODIUM LAURETH SULFATE                             50 non-null int64
DIMETHICONE                                        50 non-null int64
SODIUM BENZOATE                                    50 non-null int64
TETRASODIUM EDTA                                   50 non-null int64
SODIUM LAURYL SULFATE                              50 non-null int

In [254]:
with open('50_items_58_in_dict.pickle', 'wb') as to_write:
    pickle.dump(df2, to_write)